In [1]:
import numpy as np
import matplotlib.pylab as plt
from IPython.display import display
from ipywidgets import interact, IntSlider, FloatSlider, fixed, interactive, HBox
from SolarYield import *

def PlotPVpower(d, UTC, L, phi, i, gamma, albedo, Gh, Dh, Tamb):
    (a,h,TST, h_n, UTC_0, UTC_1) = SunPosition(d, UTC, L, phi)
    (Rs, Rd, theta) = IrradiationRatio(i, gamma, a, h, albedo, Gh[d,:], Dh[d,:])
    Gabs = GlassAbsorbtion((Gh[d,:]-Dh[d,:])*Rs, Dh[d,:]*Rd, theta)
    (Pmax, Vmin) = PV_panel([1000.], [25])
    (Pm, Vm) = PV_panel(Gabs, Tamb[d,:])
    fig, ax = plt.subplots(figsize=(8,6))
    ax.set_xlabel('hour of day (UTC)')
    ax.set_xlim(0,24)
    ax.set_xticks([0,3,6,9,12,15,18,21,24])
    ax.text(1,0.9*Pmax, 'Nominal Power: %.0f W' %Pmax)
    ax.set_ylabel('PV production (W)')
    ax.set_ylim(0,Pmax)
    plt.close(fig)
    ax.plot(UTC, Pm, 'k-', linewidth=2)
    ax.grid(True)
    return fig

(date, UTC, L, phi, T2m, RH, Gh, Gn, Dh, Ir, WS10m, WD10m, SP) = LoadTMY('./DataOnShore_be.csv')

interact(PlotPVpower,
         d=IntSlider(value=171, min=0, max=364, step=1, continuous_update=False),
         UTC=fixed(UTC), L=fixed(L), phi=fixed(phi), 
         i=FloatSlider(value=40, min=0, max=+90, step=5, continuous_update=False),
         gamma=FloatSlider(value=0, min=-90, max=+90, step=5, continuous_update=False),
         albedo=fixed(0.3), Gh=fixed(Gh), Dh=fixed(Dh), Tamb=fixed(T2m))


interactive(children=(IntSlider(value=171, continuous_update=False, description='d', max=364), FloatSlider(val…

<function __main__.PlotPVpower(d, UTC, L, phi, i, gamma, albedo, Gh, Dh, Tamb)>

In [3]:
import numpy as np
import matplotlib.pylab as plt
from IPython.display import display
from ipywidgets import interact, IntSlider, FloatSlider, fixed, interactive, HBox
from SolarYield import *

def PlotPVarray(d, UTC, L, phi, i, gamma, albedo, Gh, Dh, Tamb, Nps, Npp, Prated):
    (a,h,TST, h_n, UTC_0, UTC_1) = SunPosition(d, UTC, L, phi)
    (Rs, Rd, theta) = IrradiationRatio(i, gamma, a, h, albedo, Gh[d,:], Dh[d,:])
    Gabs = GlassAbsorbtion((Gh[d,:]-Dh[d,:])*Rs, Dh[d,:]*Rd, theta)
    (Pmax, Vmin) = PV_panel([1000.], [25])
    (Pm, Vm) = PV_panel(Gabs, Tamb[d,:])
    Ppv = PV_array(Pm, Vm, Nps, Npp, Prated)
    fig, ax = plt.subplots(figsize=(8,6))
    ax.set_xlabel('hour of day (UTC)')
    ax.set_xlim(0,24)
    ax.set_xticks([0,3,6,9,12,15,18,21,24])
    ax.set_ylabel('PV production (kW)')
    ax.set_ylim(0,Prated)
    plt.close(fig)
    ax.plot(UTC, Ppv, 'k-', linewidth=2)
    ax.grid(True)
    return fig

(date, UTC, L, phi, T2m, RH, Gh, Gn, Dh, Ir, WS10m, WD10m, SP) = LoadTMY('./DataOnShore_be.csv')

interact(PlotPVarray,
         d=IntSlider(value=171, min=0, max=364, step=1, continuous_update=False),
         UTC=fixed(UTC), L=fixed(L), phi=fixed(phi), 
         i=FloatSlider(value=40, min=0, max=+90, step=5, continuous_update=False),
         gamma=FloatSlider(value=0, min=-90, max=+90, step=5, continuous_update=False),
         albedo=fixed(0.3), Gh=fixed(Gh), Dh=fixed(Dh), Tamb=fixed(T2m),
         Nps=fixed(10), Npp=fixed(4), Prated=fixed(10.))

interactive(children=(IntSlider(value=171, continuous_update=False, description='d', max=364), FloatSlider(val…

<function __main__.PlotPVarray(d, UTC, L, phi, i, gamma, albedo, Gh, Dh, Tamb, Nps, Npp, Prated)>

In [5]:
def PVarrayYield(UTC, L, phi, i, gamma, albedo, Gh, Dh, Tamb, Nps, Npp, Prated, Textoutput=True):
    (Pmax, Vmin) = PV_panel([1000.], [25])
    Ppv_rated = Pmax*Nps*Npp*1e-3
    Epv = 0.
    tau_pv = np.zeros((365,len(UTC)), dtype=float)
    for d in range(365):
        (a,h,TST, h_n, UTC_0, UTC_1) = SunPosition(d, UTC, L, phi)
        (Rs, Rd, theta) = IrradiationRatio(i, gamma, a, h, albedo, Gh[d,:], Dh[d,:])
        Gabs = GlassAbsorbtion((Gh[d,:]-Dh[d,:])*Rs, Dh[d,:]*Rd, theta)
        ([Pmax], [Vmin]) = PV_panel([1000.], [25])
        (Pm, Vm) = PV_panel(Gabs, Tamb[d,:])
        Ppv = PV_array(Pm, Vm, Nps, Npp, Prated)
        tau_pv[d,:] = np.divide(Ppv,Ppv_rated)
        Epv += np.trapz(Ppv,UTC)
    if Textoutput:
        return 'PV produces %.0f kWh with specific production %.0f kWh/kWp' %(Epv, Epv/(Nps*Npp*Pmax*1e-3))
    else:
        return tau_pv

interact(PVarrayYield,
         UTC=fixed(UTC), L=fixed(L), phi=fixed(phi), 
         i=FloatSlider(value=40, min=0, max=+90, step=5, continuous_update=False),
         gamma=FloatSlider(value=0, min=-90, max=+90, step=5, continuous_update=False),
         albedo=fixed(0.3), Gh=fixed(Gh), Dh=fixed(Dh), Tamb=fixed(T2m),
         Nps=fixed(10), Npp=fixed(4), Prated=fixed(10.), Textoutput=fixed(True))

interactive(children=(FloatSlider(value=40.0, continuous_update=False, description='i', max=90.0, step=5.0), F…

<function __main__.PVarrayYield(UTC, L, phi, i, gamma, albedo, Gh, Dh, Tamb, Nps, Npp, Prated, Textoutput=True)>